# Alice CID categorization sheet export

The *gestão de saúde magenta* squad inputs dw weights for daly calculations manually in a google sheet. This notebook is to export the sheet, after downloading from the Google Drive and storing it in a local computer. Since the sheet is updated not very often, there is not much gain from automating the process in Airflow (*yet*).

Steps:

    - Download the google sheet DALY weights and store it in the 'redshift_import_export\external\' package folder;

    - run this notebook to upload to Redshift;
    
    - use the DW to make calculations, new tables, dashes...
***

## 0. Setup

In [22]:
#connection to redshift
from redshift_import_export.interfaces.data_interactor import DataInteractor
di = DataInteractor()
import requests


#standard packages
import warnings
warnings.filterwarnings('ignore')
import os
import collections
import pandas as pd
import numpy as np



#time packages
import datetime as dt

## 1. Import

In [23]:
df = pd.read_csv(os.path.abspath(os.path.join(os.getcwd(), os.pardir)) + '/data/external/alice_cid_categories.csv')

In [24]:
df.columns

Index(['Unnamed: 0', 'Disease.Code.Value', 'Categoria 2.0'], dtype='object')

In [25]:
df.head(10)

,Unnamed: 0,Disease.Code.Value,Categoria 2.0
0,NaN,K35,Abdome agudo
1,NaN,K37,Abdome agudo
2,NaN,K36,Abdome agudo
3,NaN,K561,Abdome agudo
4,NaN,K562,Abdome agudo
5,NaN,K564,Abdome agudo
6,NaN,K565,Abdome agudo
7,NaN,K566,Abdome agudo
8,NaN,K56,Abdome agudo
9,NaN,F101,Abuso crônico de substâncias


## 2. Transform

In [26]:
df.rename(columns={'Disease.Code.Value': 'cid', 'Categoria 2.0': 'alice_category'}, inplace = True)
df = df[['cid', 'alice_category']]
df.drop_duplicates(inplace = True)
df.head(10)

,cid,alice_category
0,K35,Abdome agudo
1,K37,Abdome agudo
2,K36,Abdome agudo
3,K561,Abdome agudo
4,K562,Abdome agudo
5,K564,Abdome agudo
6,K565,Abdome agudo
7,K566,Abdome agudo
8,K56,Abdome agudo
9,F101,Abuso crônico de substâncias


## 3. Export

In [27]:
di.redshift.insert_table(df, table_name='alice_cid_categories', if_exists='replace')

Table alice_cid_categories inserted successfully, with 2494 rows.
